In [ ]:
import os
import sys
sys.path.append("../")

import pandas as pd
import numpy as np
import datetime as dt
import re

from src.utils.parser import parse_config
from src.utils.vault import get_secrets
from src.utils.processing import downcast
import snowflake.connector

config = parse_config(os.path.abspath(os.path.join(os.getcwd(), "../src/settings.yml")))

In [ ]:
snowflake_config = config["snowflake"]
snowflake_config.update(get_secrets("snowflake"))
snowflake_ctx = snowflake.connector.connect(**snowflake_config)

In [ ]:
%%time
query = f"""
SELECT
    t0.BOXID AS boxid,
--    t0.CHANNELID AS channelid,
    YEAROFWEEKISO(t0.DATUMTIJD) AS year,
    WEEKISO(t0.DATUMTIJD) AS week,
    COUNT(DISTINCT t0.DATUMTIJD)/(4*24*7) AS COMPLETENESS
FROM {config["snowflake"]["database"]}.{config["snowflake"]["schema"]}.{config["snowflake"]["table"]} t0
WHERE t0.CHANNELID LIKE 'register://electricity/0/activepower/sumli?avg=15'
  AND t0.DATUMTIJD < DATEADD(DAY, -DAYOFWEEKISO(CURRENT_DATE), CURRENT_DATE)
--   AND t0.BOXID IN ('075.547-1', '069.509-1')
GROUP BY t0.BOXID, t0.CHANNELID, YEAROFWEEKISO(t0.DATUMTIJD), WEEKISO(t0.DATUMTIJD)
"""

df_query = (
    pd.read_sql(sql=query, con=snowflake_ctx)
    .apply(downcast, try_numeric=True, category=True)
    .sort_values(["YEAR", "WEEK"])
)
df_query

In [ ]:
bi = """ESD.000019-1
ESD.000170-1
ESD.000090-1
ESD.000354-1
ESD.000184-1
ESD.000849-2
ESD.000688-1
ESD.001109-1
ESD.000076-1
ESD.000072-1
ESD.000423-1
ESD.000076-1
ESD.000335-1
ESD.000239-1
ESD.000849-1
ESD.000219-1
ESD.000354-1
ESD.000236-1
ESD.000411-1
ESD.000123-1
ESD.000849-2
ESD.000327-1
ESD.000379-1
ESD.000360-1
ESD.001091-1
ESD.000351-1
ESD.000992-1
ESD.000085-1
ESD.000054-1
ESD.000685-1
ESD.000406-1
ESD.000087-1
ESD.000384-1
ESD.000388-1
ESD.000096-1
ESD.000384-1
ESD.000393-1
ESD.000280-1
ESD.000384-1
ESD.000266-1"""
bi.split("\n")